In [3]:
!pip3 install torchvision


[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [58]:
import pandas as pd
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, random_split

# Set device (GPU/CPU)

In [59]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters

In [60]:
num_epochs = 10
learning_rate = 0.001
batch_size = 32
num_classes = 9  # benign and malignant
weight_decay = 0.05
num_workers = 6

# Define custom dataset class

In [71]:
class SkinDiseaseDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.data_frame = csv_file
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        # self.data_frame = self.data_frame[self.data_frame['image_type'] == 'dermoscopic']
        img_name = os.path.join(self.img_dir, self.data_frame.iloc[idx, 0])
        image = Image.open(img_name+".jpg").convert('RGB')  # Open image

        label = int(self.data_frame.iloc[idx, -1])

        if self.transform:
            image = self.transform(image)

        return image, label

# Image transformations (data augmentation + normalization)

In [72]:
transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Slight zooming in and out
    transforms.RandomHorizontalFlip(),  # Horizontal flip
    transforms.RandomRotation(degrees=10),  # Mild rotation
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2),  # Adjusting lighting conditions
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load CSV and dataset

In [73]:
train_csv = '../data/metadata/train_metadata.csv'
train_data = pd.read_csv(train_csv)
train_data = train_data[train_data['image_type'] == 'dermoscopic']
test_csv = '../data/metadata/test_metadata.csv'
test_data = pd.read_csv(test_csv)
test_data = test_data[test_data['image_type'] == 'dermoscopic']
image_dir_train = '../data/train_image'
image_dir_test = '../data/test_image'

train_data['diagnosis_label'] = LabelEncoder().fit_transform(train_data['diagnosis'])
test_data['diagnosis_label'] = LabelEncoder().fit_transform(test_data['diagnosis'])

# Data splitting (80% for training, 20% for validation)
# total_size = len(train_data)
# train_size = int(0.80 * total_size)
# validation_size = int(0.20 * total_size)
# 
# train_set, validation_set = random_split(train_data, [train_size, validation_size])


train_dataset = SkinDiseaseDataset(csv_file=train_data, img_dir=image_dir_train, transform=transform)
# validation_dataset = SkinDiseaseDataset(csv_file=validation_set, img_dir=image_dir_train, transform=transform)
test_dataset = SkinDiseaseDataset(csv_file=test_data, img_dir=image_dir_test, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
# validation_loader = DataLoader(dataset=validation_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, prefetch_factor=2, persistent_workers=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is depreca

# Load a pre-trained CNN model (ResNet18)

In [74]:
model = models.mobilenet_v3_large(weights=models.MobileNet_V3_Large_Weights.DEFAULT)

# Check the output features of the last layer
feature_num = model.features[-1].out_channels

# Initially, freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Unfreeze blocks from 10 to the last block
for i in range(10, len(model.features)):
    for param in model.features[i].parameters():
        param.requires_grad = True

# Modify the classifier for Food101 dataset
model.classifier = nn.Sequential(
    nn.Linear(feature_num, 512),
    nn.ReLU(),
    nn.BatchNorm1d(512),  # batch normalization
    nn.Dropout(0.5),      # dropout rate
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.BatchNorm1d(256),  # batch normalization
    nn.Dropout(0.5),      #  dropout rate
    nn.Linear(256, 101)   # there are 101 classes
)

# Move the model to the specified device
model.to(device)

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bi

# Loss and optimizer

In [75]:
criterion = nn.CrossEntropyLoss()
params_to_update = [param for param in model.parameters() if param.requires_grad]
optimizer = torch.optim.AdamW(params_to_update, lr=learning_rate, weight_decay=weight_decay)

# Training loop

In [ ]:
# Number of epochs
num_epochs = 10
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

best_val_loss = float('inf')
patience, trials = 2, 0

for epoch in range(num_epochs):
    train_loss, train_correct, train_total = 0.0, 0, 0
    val_loss, val_correct, val_total = 0.0, 0, 0

    # Training Phase
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Validation Phase
    # model.eval()
    # with torch.no_grad():
    #     for images, labels in validation_loader:
    #         images, labels = images.to(device), labels.to(device)
    #         outputs = model(images)
    #         loss = criterion(outputs, labels)
    #         _, predicted = torch.max(outputs.data, 1)
    #         val_total += labels.size(0)
    #         val_correct += (predicted == labels).sum().item()
    #         val_loss += loss.item()

    # Average losses and accuracy
    train_loss /= len(train_loader)
    # val_loss /= len(validation_loader)
    train_accuracy = 100 * train_correct / train_total
    val_accuracy = 100 * val_correct / val_total
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracies.append(train_accuracy)
    val_accuracies.append(val_accuracy)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, '
          f'Train Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%')

    # Early Stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        trials = 0
    else:
        trials += 1
        if trials >= patience:
            print("Early stopping triggered")
            break

# Testing loop

In [10]:
model.eval()  # Set the model to evaluation mode
test_loss = 0.0
test_correct = 0
test_total = 0

with torch.no_grad():  # Disable gradient computation
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

# Calculate average test loss and accuracy
average_test_loss = test_loss / len(test_loader)
test_accuracy = 100 * test_correct / test_total

print(f'Test Loss: {average_test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.2f}%')

Test Accuracy: 51.30%


# Save the trained model

In [ ]:
torch.save(model.state_dict(), '../model/skin_disease_detection_mobilenetV3.pth')